<a href="https://colab.research.google.com/github/OnurcanKoken/Image_Processing_OpenCV/blob/main/Capture_Photo_Video_Webcam_on_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Capture Show Read Write**

# **Image Frame Photo Video Webcam**

> *`on Colab`*


---



author: Onurcan Köken

date: 4th of May, 2021

# **Contents**

1. Plot an Image by OpenCV on Colab?
2. How to use Webcam to Take a Picture or Record a Video Stream?
3. What about Directly Recording a Video Stream while you are Reading the Video Stream?
4. To be able to Save Frames of a Video Stream



# 1. Plot an Image by Opencv on Colab?

not possible to use cv2.imshow() in colab, therefor you need to use cv2_imshow() but there are other ways too, e.g. you can use PIL library 

In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow(img)

# 2. How to use Webcam to Take a Picture or Record a Video Stream?

The following code is gethered from the toolbar at the left side of Colab.
* Click on "Code Snippets" icon and Select "Camera Capture" section
* Then, click on "INSERT"




In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

You need to ALLOW colab to access your webcam

It is possible to save the taken photo

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

It is also possible to record and show video by your webcam

In [ ]:
# Adapted from: https://stackoverflow.com/a/62804023/4879610
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def record_video(filename):
  js = Javascript("""
    async function recordVideo() {
      // mashes together the advanced_outputs.ipynb function provided by Colab, 
      // a bunch of stuff from Stack overflow, and some sample code from:
      // https://developer.mozilla.org/en-US/docs/Web/API/MediaStream_Recording_API

      // Optional frames per second argument.
      const options = { mimeType: "video/webm; codecs=vp9" };
      const div = document.createElement('div');
      const capture = document.createElement('button');
      const stopCapture = document.createElement("button");
      capture.textContent = "Start Recording";
      capture.style.background = "green";
      capture.style.color = "white";

      stopCapture.textContent = "Stop Recording";
      stopCapture.style.background = "red";
      stopCapture.style.color = "white";
      div.appendChild(capture);

      const video = document.createElement('video');
      const recordingVid = document.createElement("video");
      video.style.display = 'block';

      const stream = await navigator.mediaDevices.getUserMedia({audio:true, video: true});
      // create a media recorder instance, which is an object
      // that will let you record what you stream.
      let recorder = new MediaRecorder(stream, options);
      document.body.appendChild(div);
      div.appendChild(video);
      // Video is a media element.  This line here sets the object which serves
      // as the source of the media associated with the HTMLMediaElement
      // Here, we'll set it equal to the stream.
      video.srcObject = stream;

      // from https://stackoverflow.com/questions/62529304/is-there-any-way-to-capture-live-video-using-webcam-in-google-colab#comment114644726_62804023
      video.muted = true;

      // We're inside an async function, so this await will fire off the playing
      // of a video. It returns a Promise which is resolved when playback has 
      // been successfully started. Since this is async, the function will be 
      // paused until this has started playing. 
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      // and now, just wait for the capture button to get clicked in order to
      // start recording
      await new Promise((resolve) => {
        capture.onclick = resolve;
      });
      recorder.start();
      capture.replaceWith(stopCapture);
      // use a promise to tell it to stop recording
      await new Promise((resolve) => stopCapture.onclick = resolve);
      recorder.stop();

      let recData = await new Promise((resolve) => recorder.ondataavailable = resolve);
      let arrBuff = await recData.data.arrayBuffer();
      
      // stop the stream and remove the video element
      stream.getVideoTracks()[0].stop();
      div.remove();

      let binaryString = "";
      let bytes = new Uint8Array(arrBuff);
      bytes.forEach((byte) => {
        binaryString += String.fromCharCode(byte);
      })
      return btoa(binaryString);

    }
    """)
  try:
    display(js)
    data = eval_js('recordVideo({})')
    binary = b64decode(data)
    with open(filename, "wb") as video_file:
      video_file.write(binary)
    print(
        f"Finished recording video. Saved binary under filename in current working directory: {filename}"
    )
  except Exception as err:
      # In case any exceptions arise
      print(str(err))

 remember, you need to ALLOW here, to be able to use your webcam!

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width = 600):
  # show saved video in colab.
  video_file = open(video_path, "r+b").read()

  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

In [ ]:
# video record
video_path = "test.mp4"
record_video(video_path)

<IPython.core.display.Javascript object>

Finished recording video. Saved binary under filename in current working directory: test.mp4


In [ ]:
# play the video, sound is included
show_video(video_path)

Another way of playing the video, show_video() function enough, no need to this one

In [ ]:
!pip install -U kora
from kora.drive import upload_public
url = upload_public(video_path)
# then display it
from IPython.display import HTML
HTML(f"""<video src={url} width=500 controls/>""")

# 3. What about Directly Recording a Video Strem while you are Reading the Video Strem?

I took me a *REALLY LONG TIME* to figure out how it works, but fixed the problems and works well now (in 4th of May, 2021)

might want to check this one: https://www.geeksforgeeks.org/saving-a-video-using-opencv/

or this one works fine as well: https://stackoverflow.com/questions/62072021/opencv-video-capture-using-keyboard-to-start-stop


or this one says it works for colab: https://stackoverflow.com/questions/58924926/cv2-imshow-doesnt-render-video-file-in-google-colab

In [ ]:
import cv2 as cv
import numpy as np

In [ ]:
# at first, read the video file
#video_path = "test.mp4"
video_path = "test.mp4"
# capture temporary to get width, height, fps
vcap = cv.VideoCapture(video_path)

if vcap.isOpened(): # get resulotion and fps

    width  = vcap.get(3)  # float width
    height = vcap.get(4)  # float height
    fps = vcap.get(cv.CAP_PROP_FPS) # might work or not, if it does not, use time
    
vcap.release()

# reference: https://stackoverflow.com/questions/39953263/get-video-dimension-in-python-opencv/39953739

In [ ]:
capture = cv.VideoCapture(video_path)
ret = True
# VERY IMPORTANT! IF RESULOTION IS DIFFERENT, YOU CAN NOT RECORD!
res = (int(width), int(height)) # resulotion
fourcc = cv.VideoWriter_fourcc(*'MP4V') # "*'XVID'" can be used for .avi
# fps is changeable, you can increase or decrease or use original one
output = cv.VideoWriter('video.mp4', fourcc, 30, res)

In [ ]:
while ret:
  # read each frame
  ret, frame = capture.read()
  # write each frame
  output.write(frame)
output.release()
capture.release()
cv.destroyAllWindows()

# 4. To be able to Save Frames of a Video Strem

this is a way to save images, you can change the number of images that will be saved, upto you

In [ ]:
import matplotlib.pyplot as plt
import math
import cv2

In [ ]:
count = 0
success = True

vidcap = cv2.VideoCapture(videofile)

while success:
    if (count%one_frame_each == 0):                                   # checks frame number and keeps one_frame_each          
        success,image = vidcap.read()                                 # reads next frame           
        # image = rgb2gray(image)                                     # grayscale image
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)                # convert RGB to BGR
        plt.imsave("%s/%s%d.png" % (OUTPUT_FRAMES_PATH, frame_name, count), image, cmap= plt.cm.gray) # saves images to frame folder
        print ('*', end="")
    else:
        success,image = vidcap.read()                                 # reads next frame
    count += 1                                                        # loops counter
# reference: https://colab.research.google.com/github/Giffy/CarCrashDetector/blob/master/1_Building_a_Dataset.ipynb#scrollTo=xwVJaYVnW9bd

In [ ]:
run=False
frame=0
path= "test.mp4" #some video path

def foo(event, x, y, flags, param):
    global run
    global frame
    #check which mouse button was pressed
    #e.g. play video on left mouse click
    print("aaa12")
    if event == cv2.EVENT_LBUTTONDOWN:
        run= not run
        print("aaa13")
        while run:

            frame+=1
            frame=cap.read()[1]
            cv2.imshow(window_name, frame)


    elif event == cv2.EVENT_RBUTTONDOWN:
        print("aaa14")
        pass
        #do some other stuff on right click

print("aaa1")
window_name='videoPlayer'
print("aaa2")
cv2.namedWindow(window_name)
print("aaa3")
cv2.setMouseCallback(window_name, foo)
print("aaa4")

cap=cv2.VideoCapture(path)



# > `The End`

